In [20]:
import json
from pathlib import Path
from datetime import datetime
from functools import reduce
from collections import Counter
import pymorphy2

In [31]:
raw_data = json.loads(Path("result.json").read_text())
morph = pymorphy2.MorphAnalyzer()
user_msg_freq_by_hour = {}
user_msg_type = {}
user_msg_word_freq = {}
user_msg_normalized_word_freq = {}
user_msg_count = {}
user_char_count = {}

In [16]:
def normalize_text(text):
    if type(text) == str:
        return text

    for a in text:
        if type(a) == str:
            return a

    return ""

In [24]:
def union_nd(a: dict, b: dict):
    """
    Мерджит словари. Ну то есть если у нас есть словарик
    где ключ - пользователь, а значение другой словарик,
    то мы вернем словарик склееных "словарей нижнего уровня"
    :param a:
    :param b:
    """
    if type(a.values().__iter__().__next__()) == dict:
        a = reduce(union_nd, a.values())

    if type(b.values().__iter__().__next__()) == dict:
        b = reduce(union_nd, b.values())

    res = {}

    keys = set(a.keys()).union(set(b.keys()))

    for k in keys:
        if k in a:
            res[k] = res.get(k, 0) + a[k]

        if k in b:
            res[k] = res.get(k, 0) + b[k]

    return res

In [32]:
for msg in raw_data["messages"]:
    if "text" in msg:
        msg["text"] = normalize_text(msg["text"])

    if "from" not in msg:
        continue

    if msg["from"] not in user_msg_freq_by_hour:
        user_msg_freq_by_hour[msg["from"]] = Counter()

    user_msg_freq_by_hour[msg["from"]][datetime.fromisoformat(msg["date"]).hour] += 1

    if msg["from"] not in user_msg_type:
        user_msg_type[msg["from"]] = Counter()

    if "media_type" in msg:
        user_msg_type[msg["from"]][msg["media_type"]] += 1
    else:
        user_msg_type[msg["from"]]["text"] += 1

    if msg["from"] not in user_msg_word_freq:
        user_msg_word_freq[msg["from"]] = Counter()

    if msg["from"] not in user_msg_normalized_word_freq:
        user_msg_normalized_word_freq[msg["from"]] = Counter()

    if "text" in msg:
        for token in list(map(lambda x: x.rstrip().rstrip("\n"), msg["text"].split())):
            user_msg_word_freq[msg["from"]][token] += 1
            try:
                user_msg_normalized_word_freq[msg["from"]][morph.parse(token)[0].normal_form] += 1
            except:
                continue

    if msg["from"] not in user_msg_count:
        user_msg_count[msg["from"]] = 0

    user_msg_count[msg["from"]] += 1

    if "text" in msg:
        if msg["from"] not in user_char_count:
            user_char_count[msg["from"]] = 0

        user_char_count[msg["from"]] += len(msg["text"])

In [33]:
# В этих словарях те же данные, но не отдельно по пользователю, а общие по чату
msg_freq_by_hour = reduce(union_nd, user_msg_freq_by_hour.values())
msg_type = reduce(union_nd, user_msg_type.values())
msg_word_freq = reduce(union_nd, user_msg_word_freq.values())
msg_normalized_word_freq = reduce(union_nd, user_msg_normalized_word_freq.values())